In [ ]:
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt
import os

In [ ]:
df = pd.read_csv('labels.csv')
df

In [ ]:
import xml.etree.ElementTree as xet

In [ ]:
filename = df['filepath'][428]
filename

In [ ]:
def getfilename(filename):

    filenameimage = xet.parse(filename).getroot().find('filename').text
    filepathimage = os.path.join('./cars',filenameimage)
    return filepathimage

In [ ]:
getfilename(filename)

In [ ]:
imagepath = list(df['filepath'].apply(getfilename))


In [ ]:
filepath = imagepath[0]
filepath

In [ ]:
img= cv.imread(filepath)
cv.namedWindow('example',cv.WINDOW_NORMAL)
cv.imshow('example',img)
cv.waitKey(0)
cv.destroyAllWindows()


In [ ]:
cv.rectangle(img,(226,125),(419,173),(0,225,0),3)
cv.namedWindow('example',cv.WINDOW_NORMAL)
cv.imshow('example',img)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
#data preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
labels = df.iloc[:,1:].values
labels[1]

In [ ]:
data = []
output = []

for ind in range (len(imagepath)):
    ind = 0
    image = imagepath[0]
    imgarr = cv.imread(image)
    h,w,d = imgarr.shape
    loadimage = load_img(image,target_size=(224,224))
    loadimagearr=img_to_array(loadimage)
    normloadimagearr = loadimagearr/255.0
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w, xmax/w
    nymin,nymax = ymin/h,ymax/h
    labelnorm = (nxmin,nxmax,nymin,nymax)
    data.append(normloadimagearr)
    output.append(labelnorm)

In [ ]:
x = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [ ]:
x.shape,y.shape

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,train_size=0.8,random_state=0)
xtrain.shape,xtest.shape,ytrain.shape,ytest.shape

###############################Deep learning model##############################################

In [ ]:
from tensorflow.keras.applications import MobileNetV2, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [ ]:
inceptionresnet = InceptionResNetV2(weights="imagenet",include_top=False,input_shape=(224,224,3))
inceptionresnet.trainable=False
headmodel = inceptionresnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
model = Model(inputs=inceptionresnet.input,outputs=headmodel)

In [ ]:
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

In [ ]:
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
tfb = TensorBoard('object_detection')

In [ ]:
history = model.fit(x=xtrain,y=ytrain,batch_size=5,epochs=50,
                   validation_data=(xtest,ytest),callbacks=[tfb])

In [ ]:
#to save the model in folder named models
model.save('./models/objectdetection.h5')